In [5]:
import json

import nltk

states = ["ak","al","ar","az","ca","co","ct","de","fl","ga","hi","ia","id","il",
          "in","ks","ky","la","ma","md","me","mi","mn","mo","ms","mt","nc","nd","ne","nh",
          "nj","nm","nv","ny","oh","ok","or","pa","ri","sc","sd","tn","tx","ut","va","vt",
          "wa","wi","wv","wy"]

states_names = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
'CT': 'Connecticut', 'DE': 'Delaware', 'DC': 'District of Columbia', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC':'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
'SC': 'South Carolina', 'SD': 'South Dakota', 'TN':'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
'WA': 'Washington','WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'PR': 'Puerto Rico'}

states_codes = {'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8,
'CT': 9, 'DE': 10, 'DC': 11, 'FL': 12, 'GA': 13, 'HI': 15, 'ID': 16, 'IL': 17,
'IN': 18, 'IA': 19, 'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24,
'MA': 25, 'MI': 26, 'MN': 27, 'MS': 28, 'MO': 29, 'MT': 30,
'NE': 31, 'NV': 32, 'NH': 33, 'NJ': 34, 'NM': 35, 'NY': 36, 'NC':37, 'ND': 38, 'OH': 39, 'OK': 40,
'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46, 'TN':47,
'TX': 48, 'UT': 49, 'VT': 50, 'VA': 51,
'WA': 53,'WV': 54, 'WI': 55, 'WY': 56, 'PR': 72}

afinnfile = open("../Entrega/AFINN-111.txt")
afinnDict = {} # initialize an empty dictionary
#scores_tuples = [line.decode('utf-8').split('\t') for line in afinnline]
#scores = dict((a,int(b)) for a,b in scores_tuples)

for line in afinnfile:
    term, score  = line.split("\t")  # The file is tab-delimited. "\t" means "tab character"
    afinnDict[term] = int(score)  # Convert the score to an integer.

def getState(data):
    if data["place"] != None and data["place"]["country_code"] == "US":
        state = str(data["place"]["full_name"]).lower().split(", ")
        
        if len(state) > 1:
            return state[1]

def isState(state):
    if state in states:
        return True
    return False 

def det_sent(tweet): #helper method for determining net sentiment, will use the sentiment score to appply scores to unscored words.
    terms_count = 0
    score = 0

    for word in tweet:
        stripped_word = word.lower().strip().replace('.','').replace('?','').replace('!','')

        if stripped_word in afinnDict.keys():
            terms_count += 1
            score += afinnDict[stripped_word]
        
    return terms_count, score

def cleanNotUsefulTweets():
    file = "inp.txt"
    
    linesToKeep = []
    
    with open(file, "r", encoding='utf-16') as ins:
        
        for line in ins:          
            cleanline = line.strip("'<>() ").replace('\'', '\"')
            cleanline = cleanline.replace("b\"{", "{")
            cleanline = cleanline.replace("}\"", "}")
            cleanline = cleanline.replace("\\\\", "\\")
        
            if len(cleanline) > 1: ## to avoid empty lines 
                tweet = json.loads(cleanline, encoding="utf-16")
           
                if "created_at" in tweet:
                    state = getState(tweet)

                    if isState(state):
                        if "text" in tweet:                           
                            tweetText = tweet["text"]
                            tokenizedTweet = nltk.word_tokenize(tweetText)
                            
                            score = 0
                            numberTerms = 0
                            
                            numberTerms, score = det_sent(tokenizedTweet)
                            
                            if numberTerms > 0:
                                linesToKeep.append(line)
                                
    with open(file, "w", encoding='utf-16') as ins:
        for line in linesToKeep:
            ins.write(line)

In [6]:
import json

def isTweetInSapin(tweet):
    return tweet["place"] != None and tweet["place"]["country_code"] == "ES"

def cleanNotUsefulTweetsSpain():
    file = "inpSpain.txt"

    linesToKeep = []
       
    with open(file, "r", encoding='utf-16') as ins:
        
        for line in ins:          
            cleanline = line.strip("'<>() ").replace('\'', '\"')
            cleanline = cleanline.replace("b\"{", "{")
            cleanline = cleanline.replace("}\"", "}")
            cleanline = cleanline.replace("\\\\", "\\")
        
            if len(cleanline) > 1: ## to avoid empty lines 
                tweet = json.loads(cleanline, encoding="utf-16")
           
                if "created_at" in tweet:
                    if isTweetInSapin(tweet):
                        coordinates = tweet["place"]["bounding_box"]["coordinates"][0][0]
                        lang = tweet["lang"]

                        if coordinates != None and lang != None:
                            linesToKeep.append(line)
                            
    with open(file, "w", encoding='utf-16') as ins:
        for line in linesToKeep:
            ins.write(line)

In [7]:
def append(origFile, newFile):
    origLines = []
    newLines = []
    
    with open(origFile, "r", encoding='utf-16') as lines:
        for line in lines:
            origLines.append(line)

    with open(newFile, "r", encoding='utf-16') as lines:
        for line in lines:
            if line not in origLines:
                newLines.append(line)
        
    with open(origFile, "w", encoding='utf-16') as writer:
        for line in origLines:
            writer.write(line)
            
        for line in newLines:
            writer.write(line)

In [9]:
cleanNotUsefulTweets()
cleanNotUsefulTweetsSpain()

append("../Entrega/inp.txt", "inp.txt")
append("../Entrega/inpSpain.txt", "inpSpain.txt")